# Notebook for delivering of the Airgo project

### Imports

In [1]:
import grid2op
from grid2op.Agent import TopologyGreedy, RecoPowerlineAgent
from grid2op.Runner import Runner
from grid2op.Reward import L2RPNReward

import os
import sys
from pathlib import Path

sys.path.insert(0, '../')

import src

from src.Backend.powsybl_backend import PowsyblBackend

C:\Users\rtschupp\AppData\Local\pypoetry\Cache\virtualenvs\pypowsybltest-3L0u-b9v-py3.10\lib\site-packages\grid2op\Backend\pandaPowerBackend.py:30: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	C:\Users\rtschupp\AppData\Local\pypoetry\Cache\virtualenvs\pypowsybltest-3L0u-b9v-py3.10\Scripts\python.exe -m pip install numba

  warnings.warn(


### Paths to the data

In [6]:
path = Path(os.getcwd())
root_path = path.parent.absolute()
ieee14_path = os.path.join(root_path,os.path.join("src",os.path.join("data_test","l2rpn_case14_sandbox_Pypowsybl")))
ieee1888_path = os.path.join(root_path,os.path.join("src",os.path.join("data_test","Test_1888rte")))


### Evaluation function

In [7]:
from l2rpn_baselines.utils import save_log_gif 

def evaluate(env,
             grid2op_agent,
             nb_episode=1,
             nb_process=1,
             max_steps=-1,
             verbose=False,
             save_gif=False,
             logs_path=os.path.join(root_path,"logs")):
    
    if nb_episode == 0:
        return grid2op_agent, []

    # Build runner
    runner_params = env.get_params_for_runner()
    runner_params["verbose"] = verbose
    runner = Runner(**runner_params,
                    agentClass=None,
                    agentInstance=grid2op_agent)
    
    # Run the agent on the scenarios
    if logs_path is not None:
        os.makedirs(logs_path, exist_ok=True)

    res = runner.run(path_save=logs_path,
                     nb_episode=nb_episode,
                     nb_process=nb_process,
                     max_iter=max_steps,
                     pbar=verbose,
                     )

    # Print summary
    if verbose:
        print("Evaluation summary:")
        for _, chron_name, cum_reward, nb_time_step, max_ts in res:
            msg_tmp = "chronics at: {}".format(chron_name)
            msg_tmp += "\ttotal score: {:.6f}".format(cum_reward)
            msg_tmp += "\ttime steps: {:.0f}/{:.0f}".format(nb_time_step, max_ts)
            print(msg_tmp)

    if save_gif:
        if verbose:
            print("Saving the gif of the episodes")
        save_log_gif(logs_path, res)

### Loading env and evaluation of an agent

In [9]:
env = grid2op.make(
#         ieee14_path,
        ieee1888_path,
        reward_class = L2RPNReward,
        backend = PowsyblBackend(detailed_infos_for_cascading_failures=False)
    )

evaluate(
    env,
    RecoPowerlineAgent(env.action_space),
    verbose=True,
#     save_gif = True,
    max_steps=50,
)

C:\Users\rtschupp\AppData\Local\pypoetry\Cache\virtualenvs\pypowsybltest-3L0u-b9v-py3.10\lib\site-packages\grid2op\MakeEnv\MakeFromPath.py:317: UserWarning: Impossible to load the coordinate of the substation with error: "Grid2OpException EnvError "Cannot find D:\Projets\AIRGo\OfficialRepo\pypowsybl-grid2opbackend\src\data_test\Test_1888rte\grid_layout.json. Dataset grid layout"". Expect some issue if you attempt to plot the grid.
  warnings.warn(
C:\Users\rtschupp\AppData\Local\pypoetry\Cache\virtualenvs\pypowsybltest-3L0u-b9v-py3.10\lib\site-packages\grid2op\MakeEnv\MakeFromPath.py:347: UserWarning: Dataset D:\Projets\AIRGo\OfficialRepo\pypowsybl-grid2opbackend\src\data_test\Test_1888rte\config.py doesn't have a valid graph layout. Expect some failures when attempting to plot the grid. Error was: [Errno 2] No such file or directory: 'D:\\Projets\\AIRGo\\OfficialRepo\\pypowsybl-grid2opbackend\\src\\data_test\\Test_1888rte\\grid_layout.json'
  warnings.warn(
gen vm_pu < bus min_vm_pu f

Grid2OpException: Grid2OpException "Impossible to initialize the powergrid, the powerflow diverge at iteration 0. Available information are: {'disc_lines': array([-1, -1, -1, ..., -1, -1, -1]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': False, 'is_illegal_reco': False, 'reason_alarm_illegal': None, 'reason_alert_illegal': None, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException DivergingPowerFlow DivergingPowerFlow('Isolated load')], 'rewards': {}}"